In [1]:
import sys
sys.path.append('../build')   # assume an out-of-source build
import exafmm_laplace as fmm
import numpy as np

In [2]:
fmm.__doc__

"exafmm's pybind11 module for Laplace kernel"

#### create sources and targets

In [3]:
nsrcs = 100000
ntrgs = 100000

In [4]:
src_coords = np.random.random((nsrcs, 3))
src_charges = np.random.random(nsrcs)
trg_coords = np.random.random((ntrgs, 3))

In [5]:
sources = fmm.init_sources(src_coords, src_charges)
targets = fmm.init_targets(trg_coords)

#### set FMM parameters

In [6]:
p = 10       # expansion order
ncrit = 100  # max number of bodies allow per leaf box
fmm.configure(p, ncrit)

#### build tree

In [7]:
nodes = fmm.build_tree(sources, targets)

#### build interaction lists

In [8]:
nodes = fmm.build_list()

#### check colleagues of a node (after creating the interaction list)

In [9]:
type(nodes[500].colleagues)   # list of nodes

list

In [10]:
print(nodes[500].x)

for i, colleague in enumerate(nodes[500].colleagues):
    print(i, colleague.x, colleague.level, colleague.key)

[0.218748, 0.343748, 0.406247]
0 [0.156248, 0.281248, 0.343747] 4 810
1 [0.218748, 0.281248, 0.343747] 4 814
2 [0.281249, 0.281248, 0.343747] 4 1034
3 [0.156248, 0.343748, 0.343747] 4 812
4 [0.218748, 0.343748, 0.343747] 4 816
5 [0.281249, 0.343748, 0.343747] 4 1036
6 [0.156248, 0.406249, 0.343747] 4 826
7 [0.218748, 0.406249, 0.343747] 4 830
8 [0.281249, 0.406249, 0.343747] 4 1050
9 [0.156248, 0.281248, 0.406247] 4 817
10 [0.218748, 0.281248, 0.406247] 4 821
11 [0.281249, 0.281248, 0.406247] 4 1041
12 [0.156248, 0.343748, 0.406247] 4 819
13 [0.218748, 0.343748, 0.406247] 4 823
14 [0.281249, 0.343748, 0.406247] 4 1043
15 [0.156248, 0.406249, 0.406247] 4 833
16 [0.218748, 0.406249, 0.406247] 4 837
17 [0.281249, 0.406249, 0.406247] 4 1057
18 [0.156248, 0.281248, 0.468748] 4 818
19 [0.218748, 0.281248, 0.468748] 4 822
20 [0.281249, 0.281248, 0.468748] 4 1042
21 [0.156248, 0.343748, 0.468748] 4 820
22 [0.218748, 0.343748, 0.468748] 4 824
23 [0.281249, 0.343748, 0.468748] 4 1044
24 [0.15624

Some nodes may not have all 27 colleagues. In that case, `len(colleagues)` is still 27, but the missing colleagues are filled with `None`. For example, the level 1 node `nodes[2]` only have 8 colleagues, other entries are `None`:

In [11]:
print(nodes[2].colleagues)

[None, None, None, None, None, None, None, None, None, None, None, None, <exafmm_laplace.Node object at 0x7f353c29bf48>, <exafmm_laplace.Node object at 0x7f353c29b3e8>, None, <exafmm_laplace.Node object at 0x7f353c29b880>, <exafmm_laplace.Node object at 0x7f353c29b848>, None, None, None, None, <exafmm_laplace.Node object at 0x7f353c29b810>, <exafmm_laplace.Node object at 0x7f353c29b7d8>, None, <exafmm_laplace.Node object at 0x7f353c29b688>, <exafmm_laplace.Node object at 0x7f353c29b650>, None]


#### precompute

In [12]:
fmm.precompute()

#### evaluate potentials

In [13]:
potentials = fmm.evaluate()

P2M                  : 1.7291100e-01
M2M                  : 2.0890200e-01
P2L                  : 2.0300000e-03
M2P                  : 3.1480000e-03
P2P                  : 6.5590000e-02
M2L                  : 1.2827290e+00
L2L                  : 2.8032200e-01
L2P                  : 1.8716100e-01


In [14]:
potentials[::10000]

array([6384.41543068, 7598.61228647, 8922.61785413, 7989.88122861,
       7727.27394521, 7039.23554334, 8554.48367265, 8949.34994619,
       8274.83190597, 7776.58244945])

#### check accuracy

In [15]:
fmm.check_accuracy()

-------------- Error ---------------
Potential Error      : 2.1312312e-09
Gradient Error       : 1.8006497e-07


#### update charges and run FMM iteratively

In [16]:
niters = 10
for i in range(niters):
    print('-'*10 + ' iteration {} '.format(i) + '-'*10)  # print divider between iterations
    src_charges = np.random.random(nsrcs)   # generate new random charges
    fmm.update(src_charges)      # update charges
    fmm.clear()                  # clear values
    potentials = fmm.evaluate()  # evaluate potentials
    fmm.check_accuracy()         # check accuracy

---------- iteration 0 ----------
P2M                  : 1.7140700e-01
M2M                  : 1.4644100e-01
P2L                  : 2.1080000e-03
M2P                  : 4.5310000e-03
P2P                  : 6.7304000e-02
M2L                  : 1.0163260e+00
L2L                  : 1.4114000e-01
L2P                  : 2.0792200e-01
-------------- Error ---------------
Potential Error      : 2.1043590e-09
Gradient Error       : 1.7496228e-07
---------- iteration 1 ----------
P2M                  : 1.8065100e-01
M2M                  : 2.1102900e-01
P2L                  : 3.8690000e-03
M2P                  : 3.7880000e-03
P2P                  : 6.9853000e-02
M2L                  : 1.2430870e+00
L2L                  : 2.2302400e-01
L2P                  : 2.3018100e-01
-------------- Error ---------------
Potential Error      : 2.0533422e-09
Gradient Error       : 1.7227826e-07
---------- iteration 2 ----------
P2M                  : 1.8159300e-01
M2M                  : 1.6422200e-01
P2L       